## Laboratorio 5 - Modelación y Simulación
### Ejercicio 1

Stefano Aragoni, Carol Arévalo, Luis Santos

----------------

#### Ejercicio 1 - Depredador - Presa
Debe crear una simulación de depredador-presa utilizando modelos basados en agentes. Modele varios tipos de
depredadores con diferentes comportamientos y agregue obstáculos que afecten los movimientos de los agentes.

Requisitos:
- Cree una clase Predator con diferentes subclases para diferentes tipos de depredadores.
- Modele diferentes comportamientos para distintos tipos de depredadores, como depredadores "cazadores" que persiguen activamente a sus presas y depredadores "emboscadores" que esconden y sorprenden a sus presas.
- Agregue obstáculos que los agentes deben rodear.
- Simule los movimientos de los agentes y visualice sus interacciones

----------------

#### Librerías

En esta sección se importan las librerías necesarias para el desarrollo del ejercicio. Asimismo, se definen las constantes que se utilizarán en el ejercicio.

In [145]:
import numpy as np
import matplotlib.pyplot as plt
import uuid
from PIL import Image
from IPython.display import display, Image as IPImage

map = 100
num_steps = 25
num_prey = 20
num_predators = 5
num_obs = 10

-----------

#### Clases de Agentes

En esta sección se declaraon las clases de los siguientes agentes:
- Obstacle
- Predator
    - Hunter
    - Ambusher
- Prey

In [146]:
class Obstacle():
    def __init__(self, x, y):
        self.position = np.array([x, y])

In [147]:
class Predator():
    def __init__(self, x, y, tipo):
        self.position = np.array([x, y])
        self.instance_hash = uuid.uuid4().hex
        self.type = tipo

In [148]:
class Hunter(Predator):
    def __init__(self, x, y):
        super().__init__(x, y, 'hunter')
    
    def update(self, prey_list, obstacle_list):
        # Regla 1: 
        pass

In [149]:
class Ambusher(Predator):
    def __init__(self, x, y):
        super().__init__(x, y, 'ambusher')
        self.speed = 10
    
    def update(self, prey_list, obstacle_list):
        # ------------------------------------------------------------
        # Regla 0: Presas vivas - Revisar si hay presas vivas en el mapa
        if len(prey_list) == 0:
            # No hay presas vivas, no hacer nada
            return

        # ------------------------------------------------------------
        # Regla 1: Cazar - Si hay una presa en el range de caza, cazarla
        closest_prey = min(prey_list, key=lambda prey: np.linalg.norm(self.position - prey.position))

        if np.linalg.norm(self.position - closest_prey.position) <= 10:
            # Capturar presa
            self.position = closest_prey.position
            return
        
        # ------------------------------------------------------------
        # Regla 2: Moverse al obstaculo mas cercano para emboscar
        closest_obstacle = min(obstacle_list, key=lambda obstacle: np.linalg.norm(self.position - obstacle.position))

        # Si el obstaculo esta a menos de la velocidad, moverse al obstaculo
        if np.linalg.norm(self.position - closest_obstacle.position) <= self.speed:
            # Acercarse al obstaculo en un paso pero no atravesarlo
            new_position = closest_obstacle.position + (self.position - closest_obstacle.position) / np.linalg.norm(self.position - closest_obstacle.position)

        # Si el obstaculo esta a mas de la velocidad, moverse en la direccion del obstaculo
        else:
            obstacle_direction = self.position - closest_obstacle.position
            obstacle_direction = obstacle_direction / np.linalg.norm(obstacle_direction)

            new_position = self.position + obstacle_direction * self.speed

        # ------------------------------------------------------------
        # Regla 4: Que no se salga del mapa - Restringir la nueva posición dentro de los límites del mapa
        new_position = np.clip(new_position, 0, map - 1)

        self.position = new_position

In [150]:
class Prey():
    def __init__(self, x, y):
        self.position = np.array([x, y])
        self.alive = True
        self.instance_hash = uuid.uuid4().hex
        self.speed = np.random.randint(1, 3)
    
    def update(self, predator_list, obstacle_list):

        # ------------------------------------------------------------
        # Regla 0: Que no se lo coma el depredador - Verificar si el depredador lo atrapó
        closest_predator = min(predator_list, key=lambda predator: np.linalg.norm(self.position - predator.position))
        if np.linalg.norm(self.position - closest_predator.position) < 3:
            self.alive = False
            return

        # ------------------------------------------------------------
        # Regla 1: Escapar de depredador - Huir del depredador más cercano
        closest_predator = min(predator_list, key=lambda predator: np.linalg.norm(self.position - predator.position))

        if np.linalg.norm(self.position - closest_predator.position) < 10:
            # Si el depredador está cerca, huir de él
            direction = self.position - closest_predator.position           # Calcular la dirección hacia el depredador más cercano 

        else:
            # Si el depredador está lejos, moverse o no moverse aleatoriamente
            if np.random.random() < 0.5:
                direction = np.random.random(2) * 2 - 1                      # Moverse aleatoriamente
            else:
                direction = np.array([0, 0])

        # ------------------------------------------------------------
        # Regla 2: Evitar obstáculos - Evitar el obstáculo más cercano
        closest_obstacle = min(obstacle_list, key=lambda obstacle: np.linalg.norm(self.position - obstacle.position))
        if np.linalg.norm(self.position - closest_obstacle.position) < 5:
            # Si el obstáculo está cerca, alejarse de él
            obstacle_direction = self.position - closest_obstacle.position
            obstacle_direction = obstacle_direction / np.linalg.norm(obstacle_direction)

        else:
            obstacle_direction = np.array([0, 0])

        # ------------------------------------------------------------
        # Regla 3: Alejarse de depredadores y obstáculos
        direction = (direction + obstacle_direction) / 2

        if np.linalg.norm(direction) > 0:       # Si la dirección normalizada no es cero, moverse en esa dirección
            direction_normalized = direction / np.linalg.norm(direction)
            new_position = self.position + direction_normalized * self.speed
        else:                                   # Si la dirección normalizada es cero, moverse aleatoriamente
            direction_random = np.random.random(2) * 2 - 1
            new_position = self.position + direction_random * self.speed

        # ------------------------------------------------------------
        # Regla 4: Que no se salga del mapa - Restringir la nueva posición dentro de los límites del mapa
        new_position = np.clip(new_position, 0, map - 1)

        self.position = new_position


-------

#### Creación de agentes

En esta sección se crean los agentes que se utilizarán en la simulación. Se aleatorizan las posiciones de los agentes en el espacio de simulación.

- Obstáculos

In [151]:
obstacles = []
positions = ["-1,-1"]

for i in range(num_obs):

    x = np.random.randint(0, map-5)
    y = np.random.randint(0, map-5)

    for j in range(0, 6):
        for k in range(0, 6):
            obstacles.append(Obstacle(x, y))
            positions.append(str(x) + "," + str(y))

            x += 1

        x -= 6
        y += 1

- Depredadores

In [152]:
predators = []

for i in range(0, num_predators):
    x = -1
    y = -1

    while str(x) + "," + str(y) in positions:
        x = np.random.randint(0, map-1)
        y = np.random.randint(0, map-1)

    if np.random.random() < 0.5:
        predators.append(Hunter(x, y))
    else:
        predators.append(Ambusher(x, y))

- Presas

In [153]:
prey = []

for i in range(0, num_prey):
    x = -1
    y = -1

    while str(x) + "," + str(y) in positions:
        x = np.random.randint(0, map-1)
        y = np.random.randint(0, map-1)

    prey.append(Prey(x, y))

----------------

#### Simulación

En esta sección se simula el funcionamiento de los agentes. Se definen las reglas de comportamiento de los agentes y se ejecuta la simulación.

In [154]:
list_images = []

for step in range(num_steps):
    plt.figure(figsize=(8, 8))
    plt.xlim(0, map)
    plt.ylim(0, map)
    
    prey_positions = np.array([p.position for p in prey if p.alive])
    predator_positions = np.array([p.position for p in predators])

    for obstacle in obstacles:
        plt.plot(obstacle.position[0], obstacle.position[1], 'ko')
    
    if prey_positions.shape[0] > 0:
        for predator in predators:
            predator.update(prey, obstacles)
        
        for p in prey:
            if p.alive:
                p.update(predators, obstacles)
                plt.plot(p.position[0], p.position[1], 'go')
        
        for predator in predators:
            if predator.type == 'hunter':
                plt.plot(predator.position[0], predator.position[1], 'ro')
            else:
                plt.plot(predator.position[0], predator.position[1], 'bo')
    
    plt.xlabel("x")
    plt.ylabel("y")
    plt.title("Step " + str(step))
    name = "images1/step_" + str(step) + ".jpg"
    plt.savefig(name, format="jpg")
    plt.close()
    list_images.append(name)

# Crear el GIF
images = [Image.open(path) for path in list_images]
images[0].save('predator_prey_simulation.gif', save_all=True, append_images=images[1:], loop=0, duration=200)

# Mostrar el GIF
display(IPImage("predator_prey_simulation.gif"))

------